In [1]:
import pandas as pd
import numpy as np
 
import matplotlib.pyplot as plt
%matplotlib inline

# Data Formatting

```python
train = pd.read_csv('../data/train.csv')
train.shape
weather = pd.read_csv('../data/weather.csv')
weather.shape

weather['Timestamp'] = pd.to_datetime(weather['Timestamp'])
train['Timestamp'] = pd.to_datetime(train['Timestamp'])

train_weather = pd.merge(weather, train, how = 'right', on = ['SiteId', 'Timestamp'])

train_weather['Minutes'] = [time.minute for time in train_weather['Timestamp']]
train_weather['Hour'] = [time.hour for time in train_weather['Timestamp']]
train_weather['day'] = [time.day for time in train_weather['Timestamp']]
train_weather['wday'] = [time.weekday() for time in train_weather['Timestamp']]
train_weather['month'] = [time.month for time in train_weather['Timestamp']]
train_weather['Year'] = [time.year for time in train_weather['Timestamp']]

train_weather.head()

test = pd.read_csv('../data/submission_format.csv')
test_weather = pd.merge(weather, test, how = 'right', on = ['SiteId', 'Timestamp'])
test_weather['Timestamp'] = pd.to_datetime(test_weather['Timestamp'])

test_weather['Minutes'] = [time.minute for time in test_weather['Timestamp']]
test_weather['Hour'] = [time.hour for time in test_weather['Timestamp']]
test_weather['day'] = [time.day for time in test_weather['Timestamp']]
test_weather['wday'] = [time.weekday() for time in test_weather['Timestamp']]
test_weather['month'] = [time.month for time in test_weather['Timestamp']]
test_weather['Year'] = [time.year for time in test_weather['Timestamp']]

test_weather.head()
```

In [9]:
train_weather['wday'] = [time.weekday() for time in train_weather['Timestamp']]
test_weather['wday'] = [time.weekday() for time in test_weather['Timestamp']]

In [10]:
train_weather.to_csv('../data/train_weather.csv')
test_weather.to_csv('../data/test_weather.csv')

## Read in Formatted Data

In [32]:
train_weather = pd.read_csv('../data/train_weather.csv')
test_weather = pd.read_csv('../data/test_weather.csv')

train_weather['Timestamp'] = pd.to_datetime(train_weather['Timestamp'])
test_weather['Timestamp'] = pd.to_datetime(test_weather['Timestamp'])

In [33]:
train_weather.head()

Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1.1  Timestamp  Temperature  \
0           0             0         83877.0 2014-09-03         20.2   
1           1             1         96770.0 2014-09-03         19.0   
2           2             2         83901.0 2014-09-04         22.6   
3           3             3         96797.0 2014-09-04         20.0   
4           4             4         83925.0 2014-09-05         24.7   

    Distance  SiteId   obs_id  ForecastId         Value  Minutes  Hour  day  \
0  24.889929       1   744519           1  9.096555e+05        0     0    3   
1  20.952256       1   744519           1  9.096555e+05        0     0    3   
2  24.889929       1  7627564           1  1.748273e+06        0     0    4   
3  20.952256       1  7627564           1  1.748273e+06        0     0    4   
4  24.889929       1  7034705           1           NaN        0     0    5   

   wday  month  Year  work        Date  
0     2      9  2014     1  2014-09-03  
1     2      9  2014     1  2014-09-03  
2     3      9  2014     1  2014-09-04  
3     3      9  2014     1  2014-09-04  
4     4      9  2014     1  2014-09-05

## Time Difference in Hours Column

In [14]:
train_weather['time_diff'] = train_weather['Timestamp'].diff()
train_weather['time_diff'] = [(time.total_seconds()/3600) for time in train_weather['time_diff']]

## Find averages for all buildings

In [70]:
averages = train_weather.groupby(['SiteId', 'time_diff']).mean()

In [71]:
averages = averages.reset_index()

In [72]:
averages_quarter = averages[averages['time_diff'] == 0.25]
averages_hour = averages[averages['time_diff'] == 1.0]
averages_day = averages[averages['time_diff'] == 24.0]

In [73]:
averages_quarter = averages_quarter[['SiteId', 'time_diff', 'Value']].rename(columns={'Value': 'avg_quarter'})
averages_hour = averages_hour[['SiteId', 'time_diff', 'Value']].rename(columns={'Value': 'avg_hour'})
averages_day = averages_day[['SiteId', 'time_diff', 'Value']].rename(columns={'Value': 'avg_day'})

In [74]:
averages = pd.merge(averages_quarter, averages_hour, how = 'outer', on = 'SiteId')
averages = pd.merge(averages, averages_day, how = 'outer', on = 'SiteId')

In [75]:
averages.head()

SiteId  time_diff_x   avg_quarter  time_diff_y      avg_hour  time_diff  \
0       5         0.25  25595.227516          1.0  26001.779204        NaN   
1       6         0.25  34005.855622          1.0  34487.195803        NaN   
2       8         0.25  47201.535197          1.0  48409.949416        NaN   
3       9         0.25   3357.436132          1.0   3353.627457        NaN   
4      11         0.25   2359.358924          NaN           NaN        NaN   

   avg_day  
0      NaN  
1      NaN  
2      NaN  
3      NaN  
4      NaN

In [76]:
averages = averages[['SiteId', 'avg_quarter', 'avg_hour', 'avg_day']]

In [77]:
averages.to_csv('../data/averages.csv')

In [78]:
meta = pd.read_csv('../data/metadata.csv')
meta.head()

SiteId       Surface  Sampling  BaseTemperature  MondayIsDayOff  \
0       1   1387.205119      15.0             18.0           False   
1       2   6098.278376      30.0             18.0           False   
2       3  10556.293605       5.0             18.0           False   
3       5  12541.181277      30.0             18.0           False   
4       6   9150.195373      30.0             18.0           False   

   TuesdayIsDayOff  WednesdayIsDayOff  ThursdayIsDayOff  FridayIsDayOff  \
0            False              False             False           False   
1            False              False             False           False   
2            False              False             False           False   
3            False              False             False           False   
4            False              False             False           False   

   SaturdayIsDayOff  SundayIsDayOff  
0              True            True  
1              True            True  
2              True           False  
3              True            True  
4              True            True

In [79]:
meta_new = pd.merge(meta, averages, how = 'outer', on = 'SiteId')

In [81]:
meta_new.head()

SiteId       Surface  Sampling  BaseTemperature  MondayIsDayOff  \
0       1   1387.205119      15.0             18.0           False   
1       2   6098.278376      30.0             18.0           False   
2       3  10556.293605       5.0             18.0           False   
3       5  12541.181277      30.0             18.0           False   
4       6   9150.195373      30.0             18.0           False   

   TuesdayIsDayOff  WednesdayIsDayOff  ThursdayIsDayOff  FridayIsDayOff  \
0            False              False             False           False   
1            False              False             False           False   
2            False              False             False           False   
3            False              False             False           False   
4            False              False             False           False   

   SaturdayIsDayOff  SundayIsDayOff   avg_quarter      avg_hour       avg_day  
0              True            True           NaN           NaN  4.421744e+06  
1              True            True           NaN  67524.364653           NaN  
2              True           False           NaN           NaN  6.591267e+06  
3              True            True  25595.227516  26001.779204           NaN  
4              True            True  34005.855622  34487.195803           NaN

In [103]:
patterns = []
for i in range(len(meta_new)):
    row = meta_new.iloc[i]
    weekly = list(row[4:11])
    if weekly == [0, 0, 0, 0, 0, 1, 1]:
        patterns.append('5-2')
    elif weekly == [0, 0, 0, 0, 0, 1, 0]:
        patterns.append('Sat-off')
    elif weekly == [0, 0, 0, 0, 0, 0, 1]:
        patterns.append('Sun-off')
    elif weekly == [0, 0, 0, 0, 0, 0, 0]:
        patterns.append('All-on')
    elif weekly == [0, 0, 0, 0, 1, 1, 1]:
        patterns.append('4-3')
    else:
        patterns.append('NA')

In [104]:
from collections import Counter
counter = Counter()

In [105]:
Counter(patterns)

Counter({'4-3': 1, '5-2': 252, 'All-on': 8, 'Sat-off': 1, 'Sun-off': 5})

In [106]:
meta_new['pattern'] = patterns

In [107]:
meta_new.head()

SiteId       Surface  Sampling  BaseTemperature  MondayIsDayOff  \
0       1   1387.205119      15.0             18.0           False   
1       2   6098.278376      30.0             18.0           False   
2       3  10556.293605       5.0             18.0           False   
3       5  12541.181277      30.0             18.0           False   
4       6   9150.195373      30.0             18.0           False   

   TuesdayIsDayOff  WednesdayIsDayOff  ThursdayIsDayOff  FridayIsDayOff  \
0            False              False             False           False   
1            False              False             False           False   
2            False              False             False           False   
3            False              False             False           False   
4            False              False             False           False   

   SaturdayIsDayOff  SundayIsDayOff   avg_quarter      avg_hour       avg_day  \
0              True            True           NaN           NaN  4.421744e+06   
1              True            True           NaN  67524.364653           NaN   
2              True           False           NaN           NaN  6.591267e+06   
3              True            True  25595.227516  26001.779204           NaN   
4              True            True  34005.855622  34487.195803           NaN   

   pattern  
0      5-2  
1      5-2  
2  Sat-off  
3      5-2  
4      5-2

In [108]:
meta_new = meta_new[['SiteId', 'Surface', 'avg_quarter', 'avg_hour', 'avg_day', 'pattern']]

In [110]:
meta_new.to_csv('../data/meta_augmented.csv')

# Work Day

## Training Data

In [34]:
id_list = set(train_weather['SiteId'])
wday_list = set(train_weather['wday'])

In [35]:
i = 0 
for site in id_list:
    site_row = (meta[meta['SiteId'] == site])
    print(site)
    for wd in wday_list:
        train_weather.ix[(train_weather['SiteId'] == site) & (train_weather['wday'] == wd), 'off'] = site_row.iloc[0, 4 + wd]

1
2
3
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
25
26
27
29
32
33
34
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
57
58
59
60
61
62
63
64
65
66
67
68
69
70
72
73
74
75
76
77
78
83
84
85
86
87
88
89
90
92
93
94
96
98
99
100
101
102
105
106
107
108
109
110
111
112
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
134
135
136
139
140
141
142
143
145
146
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
167
169
170
171
172
173
174
175
176
177
178
180
181
182
183
184
185
186
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
215
216
217
218
219
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
292
293
294
295
297
298
299
300
301
302
303
304
305


## Testing Data

In [37]:
id_list = set(test_weather['SiteId'])
wday_list = set(test_weather['wday'])

i = 0 
for site in id_list:
    site_row = (meta[meta['SiteId'] == site])
    print(site)
    for wd in wday_list:
        test_weather.ix[(test_weather['SiteId'] == site) & (test_weather['wday'] == wd), 'off'] = site_row.iloc[0, 4 + wd]

1
2
3
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
25
26
27
29
32
33
34
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
57
58
59
60
61
62
63
64
65
66
67
68
69
70
72
73
74
75
76
77
78
83
84
85
86
87
88
89
90
92
93
94
96
98
99
100
101
102
105
106
107
108
109
110
111
112
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
134
135
136
139
140
141
142
143
145
146
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
167
169
170
171
172
173
174
175
176
177
178
180
181
182
183
184
185
186
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
215
216
217
218
219
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
292
293
294
295
297
298
299
300
301
302
303
304
305


In [38]:
test_weather.head()

Unnamed: 0  Unnamed: 0.1  Unnamed: 0.1.1  Timestamp  Temperature  Distance  \
0           0             0             NaN 2015-08-29          NaN       NaN   
1           1             1             NaN 2015-08-30          NaN       NaN   
2           2             2             NaN 2015-08-31          NaN       NaN   
3           3             3             NaN 2015-09-01          NaN       NaN   
4           4             4             NaN 2015-09-02          NaN       NaN   

   SiteId   obs_id  ForecastId  Value  Minutes  Hour  day  wday  month  Year  \
0       1  1677832           1    0.0        0     0   29     5      8  2015   
1       1  5379616           1    0.0        0     0   30     6      8  2015   
2       1   496261           1    0.0        0     0   31     0      8  2015   
3       1  4567147           1    0.0        0     0    1     1      9  2015   
4       1  3684873           1    0.0        0     0    2     2      9  2015   

   work        Date    off  
0     1  2015-08-29   True  
1     1  2015-08-30   True  
2     1  2015-08-31  False  
3     1  2015-09-01  False  
4     1  2015-09-02  False

In [44]:
train_weather_2 = train_weather.drop(labels = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'work'], axis = 1)
test_weather_2 = test_weather.drop(labels = ['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', 'work'], axis = 1)

In [45]:
train_weather_2.head()

Timestamp  Temperature   Distance  SiteId   obs_id  ForecastId  \
0 2014-09-03         20.2  24.889929       1   744519           1   
1 2014-09-03         19.0  20.952256       1   744519           1   
2 2014-09-04         22.6  24.889929       1  7627564           1   
3 2014-09-04         20.0  20.952256       1  7627564           1   
4 2014-09-05         24.7  24.889929       1  7034705           1   

          Value  Minutes  Hour  day  wday  month  Year        Date    off  
0  9.096555e+05        0     0    3     2      9  2014  2014-09-03  False  
1  9.096555e+05        0     0    3     2      9  2014  2014-09-03  False  
2  1.748273e+06        0     0    4     3      9  2014  2014-09-04  False  
3  1.748273e+06        0     0    4     3      9  2014  2014-09-04  False  
4           NaN        0     0    5     4      9  2014  2014-09-05  False

In [46]:
test_weather_2.head()

Timestamp  Temperature  Distance  SiteId   obs_id  ForecastId  Value  \
0 2015-08-29          NaN       NaN       1  1677832           1    0.0   
1 2015-08-30          NaN       NaN       1  5379616           1    0.0   
2 2015-08-31          NaN       NaN       1   496261           1    0.0   
3 2015-09-01          NaN       NaN       1  4567147           1    0.0   
4 2015-09-02          NaN       NaN       1  3684873           1    0.0   

   Minutes  Hour  day  wday  month  Year        Date    off  
0        0     0   29     5      8  2015  2015-08-29   True  
1        0     0   30     6      8  2015  2015-08-30   True  
2        0     0   31     0      8  2015  2015-08-31  False  
3        0     0    1     1      9  2015  2015-09-01  False  
4        0     0    2     2      9  2015  2015-09-02  False

In [47]:
train_weather_2.to_csv('../data/train_weather.csv')
test_weather_2.to_csv('../data/test_weather.csv')

# Add Holidays

In [21]:
holidays = pd.read_csv('../data/holidays.csv')
holidays['Date'] = pd.to_datetime(holidays['Date'])
holidays.head()

Unnamed: 0       Date                              Holiday  SiteId
0           0 2016-01-01                             New year       1
1           1 2016-01-18  Birthday of Martin Luther King, Jr.       1
2           2 2016-02-15                Washington's Birthday       1
3           3 2016-05-30                         Memorial Day       1
4           4 2016-07-04                     Independence Day       1

## Training Data

## Testing Data

In [ ]:
for i in holidays.index:
    print(holidays.ix[i, 'Holiday'])
    day = holidays.ix[i, 'Date'].date()
    site = holidays.ix[i, 'Date']
    
    test_weather.ix[(test_weather['SiteId'] == site) & (test_weather['Date'] == day)] = 1

In [24]:
train_weather.ix[train_weather['work'] == True,  'work'] = 0
train_weather.ix[train_weather['work'] == False, 'work'] = 1

In [25]:
test_weather.ix[test_weather['work'] == True,  'work'] = 0
test_weather.ix[test_weather['work'] == False, 'work'] = 1

In [29]:
np.sum(train_weather['work'] == 1)

7762041

In [30]:
len(train_weather)

7762041

In [31]:
train_weather.to_csv('../data/train_weather.csv')
test_weather.to_csv('../data/test_weather.csv')

In [ ]:
test_weather.head()

# Fill in Missing Values? 

In [ ]:
from sklearn.preprocessing import Imputer

impute = Imputer(missing_values = 'NaN', strategy = 'median', axis = 0)

In [ ]:
df = train_weather[train_weather['SiteId'] == 5].sort_values(['Timestamp', 'Distance'])
df = df.drop_duplicates('Timestamp', keep = 'first')

In [ ]:
meta.head()

In [ ]:
sub = pd.read_csv('../data/submission_frequency.csv')
sub['hours'] = sub['ForecastPeriodNS'] / (3600 * 1e9)

In [ ]:
sub.head(20)

## Sort first by timestamp and then by weather station distance

In [ ]:
site_two = train_weather[train_weather['SiteId'] == 2].sort_values(['Timestamp', 'Distance'])

In [ ]:
site_two.head()

## Fill NA temperature values

In [ ]:
site_two['Temperature'] = site_two['Temperature'].fillna(-99)

## Drop Duplicates

In [ ]:
site_two.drop_duplicates('Timestamp', keep='first', inplace=True)

In [ ]:
site_two.head()

## Fill NA in value column

In [ ]:
site_two['Value'] = site_two['Value'].fillna(method='ffill')

## Timestamp to Float for Now

In [ ]:
site_two['Timestamp'] = [time.days * 3600 * 24 for time in (site_two['Timestamp'] - min(site_two['Timestamp']))]

# Drop Unused columns

In [ ]:
site_two = site_two.drop(columns=['Unnamed: 0', 'Distance', 'SiteId', 'ForecastId', 'Value'])

# Processing Function

In [ ]:
def process_site(site_id, data='train'):
    
    if data == 'train':
        df = train_weather[train_weather['SiteId'] == site_id].sort_values(['Timestamp', 'Distance'])
        
    else:
        df = test_weather[test_weather['SiteId'] == site_id].sort_values(['Timestamp', 'Distance'])
    
    min_date = min(train_weather[train_weather['SiteId'] == site_id]['Timestamp'])
    
    df['Temperature'] = df['Temperature'].fillna(-99)
    
    df = df.drop_duplicates('Timestamp', keep = 'first')
    
    labels = df['Value'].fillna(method='ffill')
    labels = labels.fillna(0)
    
    df = df.drop(columns = ['Unnamed: 0', 'Distance', 'SiteId', 'ForecastId', 'Value'])
    
    df['Timestamp'] = [(time.days*3600*24) for time in (df['Timestamp'] - min_date)] 
    df = df.fillna(-99)
    
    return df, labels

In [ ]:
site_one_df, labels = process_site(1)
site_one_df.head()

# Linear Model

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lin_model = LinearRegression()
lin_model.fit(site_one_df, labels)

# Submission Data

In [ ]:
test_weather_df, test_labels = process_site(1, data='test')

In [ ]:
test_weather_df.head()

## Make Predictions

In [ ]:
from itertools import chain
from IPython.display import clear_output

In [ ]:
predictions = lin_model.predict(test_weather_df)

In [ ]:
len(predictions)

In [ ]:
import sys
import time

In [ ]:
predictions = []

for id in set(test['SiteId']):
    print(id)
    
    lin_model = LinearRegression()
    train_x, train_y = process_site(id, data = 'train')
    test_x, test_y = process_site(id, data = 'test')
    
    lin_model.fit(train_x, train_y)
    predicted = lin_model.predict(test_x)
    predictions.append(predicted)
    
    clear_output()

In [ ]:
test_submit = pd.read_csv('../data/submission_format.csv')

In [ ]:
predictions_flat = list(chain(*predictions))

# Random Forest

In [ ]:
from IPython.display import clear_output
import sys

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf_predictions = []

for i, id in enumerate(set(test['SiteId'])):

    print('Percentage Complete: {:.2f}'.format(100 * i / len(set(test['SiteId']))))
    tree_reg = RandomForestRegressor(n_estimators=500)
    train_x, train_y = process_site(id, data = 'train')
    test_x, test_y = process_site(id, data = 'test')
    
    tree_reg.fit(train_x, train_y)
    predicted = lin_model.predict(test_x)
    clear_output()
    rf_predictions.append(predicted)

In [ ]:
rf_predictions = list(chain(*predictions))
test_submit = pd.read_csv('../data/submission_format.csv')
test_submit['Value'] = rf_predictions

In [ ]:
test_submit.to_csv('../submissions/second_rf.csv', index=False)

# Feature Engineering